In [ ]:
%load_ext autoreload
%autoreload 2
import glob
import pickle
import matplotlib.pyplot as plt
import numpy as np

* TODO: distance entre data et true au cours des sous-blocs pour regarder l'appentissage
* TODO: équaliser sujet par sujet
* TODO: équaliser aussi les résultats obtenus par le modèle

In [ ]:
%matplotlib inline
fig_width = 15

from aSPEM import Analysis
import bayesianchangepoint as bcp

In [ ]:
list_sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
nb_sujet = len(list_sujet)

In [ ]:
liste = []
for fname in glob.glob('data/*pkl'):
    a = fname.split('/')[1].split('.')[0].split('_')
    liste.append(a)
            
BET = {}
for x in range(len(liste)) :
    if liste[x][0]=='pari' and liste[x][1] in list_sujet:
        a = 'data/%s_%s_%s_%s.pkl'%(liste[x][0], liste[x][1],liste[x][2],liste[x][3])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            BET[liste[x][1]] = b

# Scaling Data

In [ ]:
e = Analysis(observer='AM', mode='enregistrement')
Full = e.Full_list(modes_bcp=None)

In [ ]:
scal_va_sujet, scal_va_full = {}, []
scal_bet_sujet, scal_bet_full = {}, []

va_full, bet_full = [], []
for x in list_sujet :
    bet = []
    for block in range(BET[x]['N_blocks']) :
        bet.extend((BET[x]['results'][:,block]+1)/2)
    
    bet_full.extend(bet)
    va_full.extend(list(Full['va'][Full.sujet==x]))
    
    scal_bet_sujet[x] = np.sort(bet)
    scal_va_sujet[x] = np.sort(list(Full['va'][Full.sujet==x]))

scal_va_full = np.sort(va_full)
scal_bet_full = np.sort(bet_full)

In [ ]:
new_proba_full = np.linspace(0,1,len(scal_va_full))

new_va_sujet, new_va_full = {}, {}
new_bet_sujet, new_bet_full = {}, {}


for x in list_sujet :
    N_trials = BET[x]['N_trials']
    
    new_proba_sujet = np.linspace(0, 1, len(scal_va_sujet[x]))
    va_sujet = list(Full['va'][Full.sujet==x])
    
    
    nb_trial = 0
    
    new_va_sujet[x], new_va_full[x] = [], []
    new_bet_sujet[x], new_bet_full[x] = [], []
    
    for block in range(3) :
        new_va_sujet[x].append([])
        new_va_full[x].append([])
        
        new_bet_sujet[x].append([])
        new_bet_full[x].append([])
        
        
        va = va_sujet[nb_trial : N_trials + nb_trial*block]
        bet = (BET[x]['results'][:,block]+1)/2
        
        
        for trial in range(len(va)) :
            p = []
            for t in range(len(scal_va_sujet[x])) :
                if scal_va_sujet[x][t] == va[trial] :
                    p.append(new_proba_sujet[t])
            new_va_sujet[x][block].append(np.mean(p))
            
            p = []
            for t in range(len(scal_va_full)) :
                if scal_va_full[t] == va[trial] :
                    p.append(new_proba_full[t])
            new_va_full[x][block].append(np.mean(p))
       

        for trial in range(len(bet)) :
            p = []
            for t in range(len(scal_bet_sujet[x])) :
                if scal_bet_sujet[x][t] == bet[trial] :
                    p.append(new_proba_sujet[t])
            new_bet_sujet[x][block].append(np.mean(p))

            p = []
            for t in range(len(scal_bet_full)) :
                if scal_bet_full[t] == bet[trial] :
                    p.append(new_proba_full[t])
            new_bet_full[x][block].append(np.mean(p))
    
    
        nb_trial = nb_trial + N_trials

# Variation de h

In [ ]:
from lmfit import  Model, Parameters
from lmfit import minimize

In [ ]:
def fct_BCP(x, h) :

    p_bar, r_bar, beliefs = bcp.inference(x, h=h, p0=.5, r0=1.)
    p_hat, r_hat = bcp.readout(p_bar, r_bar, beliefs, mode='mean', p0=.5, fixed_window_size=40)#, mode='mean')
    
    return p_hat

In [ ]:
def KL_distance(p_data, p_hat):
    distance = p_hat * np.log2(p_hat) - p_hat * np.log2(p_data + 1.*(p_data==0.))
    distance += (1-p_hat) * np.log2(1-p_hat) - (1-p_hat) * np.log2(1-p_data + 1.*(p_data==1.))
    return distance

def residual(params, x, data):
    h = params['h']
    model = fct_BCP(x, h)
    
    return KL_distance(data, model)

def fit(h, x, bet, va):
    
    x = np.array(x)
    bet = np.array(bet)
    va = np.array(va)

    params = Parameters()
    params.add('h', value=h, min=0, max=1)

    result_res = minimize(residual, params, args=(x, bet), nan_policy='omit')
    result_v_ant = minimize(residual, params, args=(x, va), nan_policy='omit')
    
    h_bet = result_res.params['h'].value
    h_va = result_v_ant.params['h'].value
    
    return h_bet, h_va

In [ ]:
def fit_h(new_bet, new_va) :
    
    h_bet, h_va = {}, {}

    for l in ['pause', 'block', 'sujet'] :
        h_bet[l], h_va[l] = [], []

    for x, sujet in enumerate(list_sujet) :

        print(sujet)

        prob_sujet, bet_sujet, v_anti_sujet = [], [], []

        p = BET[sujet]['p']
        tau = BET[sujet]['N_trials']/5.
        h = 1./tau 

        for l in ['block', 'pause'] :
            h_bet[l].append([])
            h_va[l].append([])

        for block in range(BET[sujet]['N_blocks']):
            
            bet = new_bet[sujet][block]
            va = new_va[sujet][block]

            prob_block = p[:, block, 0]
            h_b, h_v = fit(h, prob_block, bet, va)

            h_bet['block'][x].append(h_b)
            h_va['block'][x].append(h_v)

            prob_sujet.extend(p[:, block, 0])
            bet_sujet.extend(bet)
            v_anti_sujet.extend(va)
            #----------------------------------------------------
            # Pour chaque pause !
            #----------------------------------------------------
            liste = [0,50,100,150,200]
            for a in range(len(liste)-1) :
                va_p = va[liste[a]:liste[a+1]]
                bet_p = bet[liste[a]:liste[a+1]]
                prob_pause = p[liste[a]:liste[a+1], block, 0]

                h_b, h_v = fit(h, prob_pause, bet_p, va_p)

                h_bet['pause'][x].append(h_b)
                h_va['pause'][x].append(h_v)


        h_b, h_v = fit(h, prob_sujet, bet_sujet, v_anti_sujet)
        h_bet['sujet'].append(h_b)
        h_va['sujet'].append(h_v)
    return h_bet, h_va

In [ ]:
h_bet_full, h_va_full = fit_h(new_bet_full, new_va_full)

In [ ]:
h_bet_sujet, h_va_sujet = fit_h(new_bet_sujet, new_va_sujet)

## evaluating the best hazard rate using a likelihood score between the predicted biases using different values of the hazard rate

In [ ]:
def plot_evaluating(ax, h_eval, data, bino, p, mode):

    Jeffreys = True
    p0, r0 = .5, 2. - Jeffreys
    
    distance = np.zeros(N_trials)
    p_hat = np.zeros(N_trials)
    p_low, p_sup = np.zeros_like(distance), np.zeros_like(distance)
    
    
    p_bar, r, beliefs = bcp.inference(bino, h=h_eval, p0=p0, r0=r0, verbose=False)
    p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode=mode, p0=p0)
    from scipy.stats import beta
    for trial in range(N_trials):
        a = p_hat[trial]*(r_hat[trial])
        b = (1-p_hat[trial])*(r_hat[trial])
        rv = beta(a, b)
        p_low[trial], p_sup[trial] = rv.ppf([.05, .95])
        distance[trial] = KL_distance(data[trial], p_hat[trial])

    distance = np.exp(-distance/np.median(distance))

    trials = range(len(data))
    ax.plot(trials, data, c='k', lw=1, alpha=.9)
    ax.plot(trials, p_hat, 'r', lw=1, alpha=.5)
    ax.scatter(trials, p_hat, c=distance, vmin=distance.min(), vmax=distance.max(), alpha=1., cmap=plt.get_cmap('RdGy'))  # coolwarm

    ax.fill_between(trials, np.zeros_like(p), p, lw=.5, alpha=0.2, facecolor='k', step='pre')
    ax.fill_between(trials, p_sup, p_low, lw=.2, alpha=0.2, facecolor='r', step='mid')
    ax.plot(trials, p_sup, 'r--', lw=1, alpha=.9)
    ax.plot(trials, p_low, 'r--', lw=1, alpha=.9)
    ax.set_ylim(-0.1, 1.1)
    
    return ax

In [ ]:
def find_best_h(ax, data, bino, h_sujet, mode, N_scan):
    
    Jeffreys = True
    p0, r0 = .5, 2. - Jeffreys
    
    hs = np.logspace(-1, 1, N_scan, base=N_scan)/N_scan

    full_distance = np.zeros(N_scan)
    for i_scan, h_ in enumerate(hs):
        p_bar, r, beliefs = bcp.inference(bino, h=h_, p0=p0, r0=r0, verbose=False)
        p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode=mode, p0=p0)
        d_ = KL_distance(np.array(data), p_hat)

        full_distance[i_scan] = np.mean(d_)

    ax.plot(hs, full_distance, 'k')
    ax.vlines(h_sujet, ymin=np.nanmin(full_distance), ymax=np.nanmax(full_distance), colors='k', linestyle='--', lw=2)

    h_opt = hs[np.argmin(full_distance)]
    ax.vlines(h_opt, ymin=np.nanmin(full_distance), ymax=np.nanmax(full_distance), colors='r', linestyle='--', lw=2)
    ax.set_xscale("log")
    ax.set_ylim(0)
    ax.set_xlabel('Hazard rate')
    ax.set_ylabel('distance')
    return ax, h_opt

In [ ]:
import matplotlib.gridspec as gridspec

def evaluating (data_sujet, data_full, h_sujet, h_full):
    
    
    for x, sujet in enumerate(list_sujet) :
        print(sujet)

        fig = plt.figure(figsize=(30,30/2))
        axs = gridspec.GridSpec(3, 2, hspace=0.2)

        mode = 'mean'

        N_trials, N_blocks = BET[sujet]['N_trials'], BET[sujet]['N_blocks']

        tau_true = 40. # N_trials/10.
        h_true = 1./tau_true
  
        for y in range(2) :
            if y==0 :
                data_eval, h_eval = data_sujet[sujet], h_sujet['block'][x]
                titre = 'scalling sujet'
            else :
                data_eval, h_eval = data_full[sujet], h_full['block'][x]
                titre = 'scalling full'
            print(titre)
            
            nb_trial = 0
            for block in range(N_blocks) :
                gs1 = gridspec.GridSpecFromSubplotSpec(2, 3, subplot_spec=axs[2*block+y], wspace=0.5)
                ax1 = plt.Subplot(fig, gs1[0,:-1])
                ax2 = plt.Subplot(fig, gs1[:,-1])
                ax3 = plt.Subplot(fig, gs1[1,:-1])
                fig.add_subplot(ax1)
                fig.add_subplot(ax2)
                fig.add_subplot(ax3)


                data = data_eval[block]
                h_fit = h_eval[block]

                bino = np.array(list(Full['bino'][Full.sujet==sujet])[nb_trial:N_trials+nb_trial*block])
                p = np.array(list(Full['proba'][Full.sujet==sujet])[nb_trial:N_trials+nb_trial*block])

                ax1 = plot_evaluating(ax1, h_fit, data, bino, p, mode)
                ax1.set_ylabel('h_fit=%.5f'%(h_fit))
                if block==0 :
                    ax1.set_title(titre)

                N_scan = 100

                ax2, h_opt = find_best_h(ax2, data, bino, h_fit, mode, N_scan)

                ax3 = plot_evaluating(ax3, h_opt, data, bino, p, mode)
                ax3.set_ylabel('h_opt=%.5f'%(h_opt))

                print('block', block, '--> tau_fit=%.1f ,'%(1/h_fit), 'h_fit=%.5f ,'%(h_fit), 'tau_opt=%.1f ,'%(1/h_opt), 'h_opt=%.5f'%(h_opt))


                nb_trial = nb_trial+N_trials

        plt.show() 

### BET

In [ ]:
evaluating (new_bet_sujet, new_bet_full, h_bet_sujet, h_bet_full)

### VA

In [ ]:
evaluating (new_va_full, h_va)